In [1]:
import os
import requests
import csv

def download_and_split_file(url, download_dir, user_filename, user_extension):
  """Downloads the file, splits if larger than 100MB, and adds part suffixes ("part_x") before the extension.
  Also writes filename and url to a CSV."""

  # Define variables (moved to the top for clarity)
  max_part_size = 90 * 1024 * 1024

  # Download the file
  response = requests.get(url, stream=True)

  if response.status_code == 200:
    # Construct filename with part suffix if needed
    part_number = 1
    filename = f"{user_filename}_part_{part_number}{user_extension}"

    # Download and split the file
    downloaded_size = 0
    with open(os.path.join(download_dir, filename), 'wb') as f:  # Open with full path
      for chunk in response.iter_content(1024):
        f.write(chunk)
        downloaded_size += len(chunk)

        # Check if part size limit is reached or end of file is reached
        if downloaded_size >= max_part_size or chunk == b'':
          f.close()  # Close the current file

          if downloaded_size >= max_part_size:
            # Create a new file for the next part
            part_number += 1
            filename = f"{user_filename}_part_{part_number}{user_extension}"
            f = open(os.path.join(download_dir, filename), 'wb')
            downloaded_size = 0  # Reset downloaded size for next part

    # Write filename and url to CSV (assuming CSV exists)
    csv_filepath = os.path.join(download_dir, "downloaded_files.csv")  # Replace with your CSV path
    try:
      # Open in append mode to add a new row
      with open(csv_filepath, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([user_filename, url])
      print(f"Filename and URL written to CSV: {csv_filepath}")
    except FileNotFoundError:
      print(f"CSV file not found: {csv_filepath}")

    # Print successful download message (added back)
    print(f"File downloaded successfully: {os.path.join(download_dir, filename)}")

  else:
    print(f"Error downloading file: {response.status_code}")

# Download and write to CSV
user_filename = "ConvensionalRenewablePowerPlants_EU"
user_extension = ".csv"
url = "https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv"
download_dir = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants"
download_and_split_file(url, download_dir, user_filename, user_extension)


Filename and URL written to CSV: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/downloaded_files.csv
File downloaded successfully: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ConvensionalRenewablePowerPlants_EU_part_1.csv
